In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("train.csv")
df.head()

,label,youtube_id,time_start,time_end,split,is_cc
0,abseiling,-3B32lodo2M,59,69,train,0
1,abseiling,-7kbO0v4hag,107,117,train,0
2,abseiling,-bwYZwnwb8E,13,23,train,0
3,abseiling,-Cv3NwxG_8g,87,97,train,0
4,abseiling,-hLv_HL6UhY,151,161,train,0


### Check Unique Action Labels

In [4]:
# List all unique action labels
unique_labels = df['label'].unique()
print(f"Total unique classes: {len(unique_labels)}")
unique_labels[:30]

Total unique classes: 400


array(['abseiling', 'air drumming', 'answering questions', 'applauding',
       'applying cream', 'archery', 'arm wrestling', 'arranging flowers',
       'assembling computer', 'auctioning', 'baby waking up',
       'baking cookies', 'balloon blowing', 'bandaging', 'barbequing',
       'bartending', 'beatboxing', 'bee keeping', 'belly dancing',
       'bench pressing', 'bending back', 'bending metal',
       'biking through snow', 'blasting sand', 'blowing glass',
       'blowing leaves', 'blowing nose', 'blowing out candles',
       'bobsledding', 'bookbinding'], dtype=object)

### Search for Relevant Keywords

In [5]:
keywords = ["throw", "put", "drop", "walk", "stand"]

# Filter rows containing these keywords
subset = df[df['label'].str.contains('|'.join(keywords), case=False)]

print(f"Found {len(subset)} matching clips")
subset[['youtube_id', 'time_start', 'time_end', 'label']].head(20)

Found 11572 matching clips


,youtube_id,time_start,time_end,label
5301,-nDK9fE_CNE,505,515,assembling computer
5302,-Q5qc7aDMQI,910,920,assembling computer
5303,-sLxWIySYdw,258,268,assembling computer
5304,-w8WPKfiHB4,657,667,assembling computer
5305,-ZEFTYa0ViE,47,57,assembling computer
5306,06SM4ZN_maw,1390,1400,assembling computer
5307,088E5GOX3xU,64,74,assembling computer
5308,0bUghCx9iso,447,457,assembling computer
5309,0cNBFhUAMgs,212,222,assembling computer
5310,0Tyx94yZV4I,211,221,assembling computer


In [7]:
# Unique action classes found with these keywords
matched_classes = subset['label'].unique()
print("Matched classes:", matched_classes)

Matched classes: ['assembling computer' 'catching or throwing baseball'
 'catching or throwing frisbee' 'catching or throwing softball'
 'drop kicking' 'golf putting' 'hammer throw' 'javelin throw'
 'riding or walking with horse' 'shot put' 'throwing axe' 'throwing ball'
 'throwing discus' 'using computer' 'walking the dog']


✅ Useful:

- "catching or throwing baseball"

- "throwing ball"

- "walking the dog"

❌ Irrelevant but keyword-matched:

- "assembling computer" (because of "put")

- "using computer" (same reason)

- "golf putting" (sports putting, not disposal putting)

In [8]:
# Keep only classes that make sense for "disposal-like" actions
relevant_classes = [
    "throwing ball",
    "catching or throwing baseball",
    "catching or throwing frisbee",
    "catching or throwing softball",
    "drop kicking",
    "walking the dog"
]

subset = df.loc[df['label'].isin(relevant_classes),
                ['youtube_id', 'time_start', 'time_end', 'label']]

print(f"Final dataset size: {len(subset)} clips")
subset.head(20)

Final dataset size: 4255 clips


,youtube_id,time_start,time_end,label
30295,-gswhyjdG-A,0,10,catching or throwing baseball
30296,-i9n9qVTlmk,21,31,catching or throwing baseball
30297,-jIBrJAYBeg,1,11,catching or throwing baseball
30298,-JLmaEHB_GI,277,287,catching or throwing baseball
30299,-sKTm-mjHYo,13,23,catching or throwing baseball
30300,-_o1wewuNhU,127,137,catching or throwing baseball
30301,-_ZMybcaUOw,374,384,catching or throwing baseball
30302,0-2GOSXJUe0,0,10,catching or throwing baseball
30303,08Rk0r1zVyk,44,54,catching or throwing baseball
30304,08sAVLdsWI4,13,23,catching or throwing baseball


In [11]:
subset.to_csv("subset.csv", index=False)
print("Subset saved! Total rows:", len(subset))

Subset saved! Total rows: 4255


In [9]:
!pip install yt-dlp


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os
from yt_dlp import YoutubeDL

# Create folder
os.makedirs("greenify_videos", exist_ok=True)

# yt-dlp options
ydl_opts = {
    'format': 'mp4',
    'outtmpl': 'greenify_videos/%(id)s.%(ext)s',  # save as video_id.mp4
    'quiet': False,
    'ignoreerrors': True
}

# Download subset
with YoutubeDL(ydl_opts) as ydl:
    for vid in subset['youtube_id']:
        url = f"https://www.youtube.com/watch?v={vid}"
        try:
            ydl.download([url])
        except Exception as e:
            print(f"Failed for {vid}: {e}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=-gswhyjdG-A
[youtube] -gswhyjdG-A: Downloading webpage
[youtube] -gswhyjdG-A: Downloading tv simply player API JSON
[youtube] -gswhyjdG-A: Downloading tv client config
[youtube] -gswhyjdG-A: Downloading player b66835e2-main
[youtube] -gswhyjdG-A: Downloading tv player API JSON
[info] -gswhyjdG-A: Downloading 1 format(s): 18
[download] Destination: greenify_videos\-gswhyjdG-A.mp4
[download] 100% of    1.60MiB in 00:00:32 at 50.99KiB/s  
[youtube] Extracting URL: https://www.youtube.com/watch?v=-i9n9qVTlmk
[youtube] -i9n9qVTlmk: Downloading webpage
[youtube] -i9n9qVTlmk: Downloading tv simply player API JSON
[youtube] -i9n9qVTlmk: Downloading tv client config
[youtube] -i9n9qVTlmk: Downloading tv player API JSON


ERROR: [youtube] -i9n9qVTlmk: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


[youtube] Extracting URL: https://www.youtube.com/watch?v=-jIBrJAYBeg
[youtube] -jIBrJAYBeg: Downloading webpage
[youtube] -jIBrJAYBeg: Downloading tv simply player API JSON
[youtube] -jIBrJAYBeg: Downloading tv client config
[youtube] -jIBrJAYBeg: Downloading tv player API JSON
[info] -jIBrJAYBeg: Downloading 1 format(s): 18
[download] Destination: greenify_videos\-jIBrJAYBeg.mp4
[download] 100% of   88.05KiB in 00:00:04 at 18.12KiB/s  
[youtube] Extracting URL: https://www.youtube.com/watch?v=-JLmaEHB_GI
[youtube] -JLmaEHB_GI: Downloading webpage
[youtube] -JLmaEHB_GI: Downloading tv simply player API JSON
[youtube] -JLmaEHB_GI: Downloading tv client config
[youtube] -JLmaEHB_GI: Downloading tv player API JSON
[info] -JLmaEHB_GI: Downloading 1 format(s): 18
[download] Destination: greenify_videos\-JLmaEHB_GI.mp4
[download]  27.4% of   35.95MiB at   30.80KiB/s ETA 14:27

[download] Got error: The read operation timed out



[youtube] Extracting URL: https://www.youtube.com/watch?v=-sKTm-mjHYo
[youtube] -sKTm-mjHYo: Downloading webpage
[youtube] -sKTm-mjHYo: Downloading tv simply player API JSON
[youtube] -sKTm-mjHYo: Downloading tv client config
[youtube] -sKTm-mjHYo: Downloading tv player API JSON
[info] -sKTm-mjHYo: Downloading 1 format(s): 18
[download] Destination: greenify_videos\-sKTm-mjHYo.mp4
[download] 100% of    1.79MiB in 00:00:43 at 41.87KiB/s    
[youtube] Extracting URL: https://www.youtube.com/watch?v=-_o1wewuNhU
[youtube] -_o1wewuNhU: Downloading webpage
[youtube] -_o1wewuNhU: Downloading tv simply player API JSON
[youtube] -_o1wewuNhU: Downloading tv client config
[youtube] -_o1wewuNhU: Downloading tv player API JSON
[info] -_o1wewuNhU: Downloading 1 format(s): 18
[download] Sleeping 1.00 seconds as required by the site...
[download] Destination: greenify_videos\-_o1wewuNhU.mp4
[download]  79.0% of   12.38MiB at   60.55KiB/s ETA 00:43

...

[youtube] Extracting URL: https://www.youtube.com/watch?v=LLAEGU4-W3M
[youtube] LLAEGU4-W3M: Downloading webpage
[youtube] LLAEGU4-W3M: Downloading tv simply player API JSON
[youtube] LLAEGU4-W3M: Downloading tv client config
[youtube] LLAEGU4-W3M: Downloading tv player API JSON
[info] LLAEGU4-W3M: Downloading 1 format(s): 18
[download] Sleeping 2.00 seconds as required by the site...
[download] Destination: greenify_videos\LLAEGU4-W3M.mp4
[download] 100% of  700.94KiB in 00:00:10 at 66.31KiB/s  
[youtube] Extracting URL: https://www.youtube.com/watch?v=LmkqNht8NCM
[youtube] LmkqNht8NCM: Downloading webpage
[youtube] LmkqNht8NCM: Downloading tv simply player API JSON
[youtube] LmkqNht8NCM: Downloading tv client config
[youtube] LmkqNht8NCM: Downloading tv player API JSON
[info] LmkqNht8NCM: Downloading 1 format(s): 18
[download] Destination: greenify_videos\LmkqNht8NCM.mp4
[download] 100% of    5.42MiB in 00:00:05 at 963.86KiB/s 
[youtube] Extracting URL: https://www.youtube.com/watch?

ERROR: [youtube] LUc5uppqib0: Video unavailable


[youtube] Extracting URL: https://www.youtube.com/watch?v=lv9a8by_oqc
[youtube] lv9a8by_oqc: Downloading webpage
[youtube] lv9a8by_oqc: Downloading tv simply player API JSON
[youtube] lv9a8by_oqc: Downloading tv client config
[youtube] lv9a8by_oqc: Downloading tv player API JSON
[info] lv9a8by_oqc: Downloading 1 format(s): 18
[download] Sleeping 1.00 seconds as required by the site...
[download] Destination: greenify_videos\lv9a8by_oqc.mp4
[download] 100% of   14.85MiB in 00:02:01 at 124.90KiB/s    
[youtube] Extracting URL: https://www.youtube.com/watch?v=LVv0JoAXjns
[youtube] LVv0JoAXjns: Downloading webpage
[youtube] LVv0JoAXjns: Downloading tv simply player API JSON
[youtube] LVv0JoAXjns: Downloading tv client config
[youtube] LVv0JoAXjns: Downloading tv player API JSON
[info] LVv0JoAXjns: Downloading 1 format(s): 18
[download] Sleeping 1.00 seconds as required by the site...
[download] Destination: greenify_videos\LVv0JoAXjns.mp4
[download] 100% of    1.11MiB in 00:00:15 at 73.22K

[youtube] M-SucFfaemc: Downloading initial data API JSON
[youtube] M-SucFfaemc: Downloading tv simply player API JSON
[youtube] M-SucFfaemc: Downloading tv client config
[youtube] M-SucFfaemc: Downloading tv player API JSON
[info] M-SucFfaemc: Downloading 1 format(s): 18
[download] Destination: greenify_videos\M-SucFfaemc.mp4
[download] 100% of   22.49MiB in 00:03:51 at 99.34KiB/s    
[youtube] Extracting URL: https://www.youtube.com/watch?v=m03-rhZhJeM
[youtube] m03-rhZhJeM: Downloading webpage
[youtube] m03-rhZhJeM: Downloading tv simply player API JSON
[youtube] m03-rhZhJeM: Downloading tv client config
[youtube] m03-rhZhJeM: Downloading tv player API JSON
[info] m03-rhZhJeM: Downloading 1 format(s): 18
[download] Destination: greenify_videos\m03-rhZhJeM.mp4
[download] 100% of   28.87MiB in 00:04:06 at 120.09KiB/s    
[youtube] Extracting URL: https://www.youtube.com/watch?v=M1c-RYCGwf8
[youtube] M1c-RYCGwf8: Downloading webpage
[youtube] M1c-RYCGwf8: Downloading tv simply player AP

KeyboardInterrupt: 